# Homework to Session 9. Serverless Deep Learning

## 1. Install TensorFlow Lite

In [1]:
! pip install tflite_runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.1 MB/s eta 0:00:0000:01


## 2. Loading initial model and converting to \*.tflite format

In [4]:
import urllib.request
import tensorflow as tf
from tensorflow import keras

In [5]:
url = 'https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5'
filename = 'dino_dragon_10_0.899.h5'
urllib.request.urlretrieve(url, filename)

('dino_dragon_10_0.899.h5', <http.client.HTTPMessage at 0x7fa1f43f9a10>)

In [6]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

2022-11-27 20:16:04.561980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 20:16:04.699025: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 20:16:04.700864: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 20:16:04.704132: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dino-dragon-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

2022-11-27 20:16:18.030126: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp59u04wv7/assets


2022-11-27 20:16:18.884680: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-11-27 20:16:18.884729: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-11-27 20:16:18.885690: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp59u04wv7
2022-11-27 20:16:18.888783: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-11-27 20:16:18.888836: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmp59u04wv7
2022-11-27 20:16:18.896702: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-11-27 20:16:19.017062: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmp59u04wv7
2022-11-27 20:16:19.030677: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 145013 

### Q1. What's the size of the converted model?

Answer: 43 Mb (closest to 42.8 Mb)

## 3. Load \*.tflite model

In [1]:
import tflite_runtime.interpreter as tflite

In [2]:
interpreter = tflite.Interpreter(model_path='dino-dragon-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Q2. What's the output index for this model?

In [3]:
output_index

13

Answer: 13

## 4. Load and preprocess image to predict

In [4]:
! pip install pillow

In [4]:
import numpy as np

In [17]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_image(x):
    x = np.true_divide(x, 255, dtype=np.float32)    # same as "x /= 255"
    return x

In [18]:
# load an image

url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'
img = download_image(url)

In [19]:
# prepare image (rescale)

target_size = 150
img = prepare_image(img, (target_size, 150))

# preprocess image

x = np.array(img, dtype='float32')
X = np.array([x])
X = preprocess_image(X)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  app.launch_new_instance()


### Q3. After the pre-processing, what's the value in the first pixel, the R channel?

In [20]:
# RGB-values for 1-st pixel

X[0][0][0]

array([0.5529412 , 0.31764707, 0.1764706 ], dtype=float32)

Answer: 0.5529412

## 5. Prediction

### Q4. Now let's apply this model to this image. What's the output of the model?

In [21]:
import tflite_runtime.interpreter as tflite

In [22]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

preds

array([[0.82448614]], dtype=float32)

In [27]:
if preds[0][0] <= 0.5:
    print('dino')
else:
    print('dragon')

dragon


Answer: 0.82448614

## 6. Convert a notebook to script

In [29]:
! jupyter nbconvert --to script homework9-notebook.ipynb

[NbConvertApp] Converting notebook homework9-notebook.ipynb to script
[NbConvertApp] Writing 2918 bytes to homework9-notebook.py
